In [1]:
from sqlalchemy import create_engine
from sqlalchemy.sql.ddl import CreateSchema
from tablas_mfm import *
import numpy as np
import pandas as pd
import generador_filas as gf


In [2]:
#Devuelve una tabla con nombre "nombre"
def string_to_table(nombre):
    for tb in metadata.sorted_tables:
        if tb.name == nombre:
            return tb
    raise Exception("la tabla "+nombre+" no existe")


In [3]:
#Copia la tabla "nombreTabla" de la BBDD "dbO" a la BBDD "dbD"
def copiarTabla(dbO,dbD, nombreTabla):
    if not isinstance(nombreTabla,Table) :
        raise Exception("la tabla "+nombreTabla+" no existe")
    else :
        result = dbO.execute(nombreTabla.select())
        column=nombreTabla.columns.keys()
        filas=[]
        filasmax=10000
        num_filas=0
        for r in result:
            fila = {}
            for i in range(len(column)):
                fila.update({column[i] : r[i]})
            filas.append(fila)
            num_filas+=1
            if num_filas > filasmax:
                insercion = nombreTabla.insert().values(filas)
                dbD.execute(insercion)
                filas = []
                num_filas=0
        result.close()
        insercion = nombreTabla.insert().values(filas)
        dbD.execute(insercion)


In [4]:
#Crea las conexiones con las BBDD origen y destino
#db_destino_string = ""
#db_origen_string = ""

dbDestino = create_engine(db_destino_string)
dbOrigen = create_engine(db_origen_string)

dbDes=dbDestino.connect()
dbOr=dbOrigen.connect()

In [5]:
#Crea el esquema para las tablas en la BBDD destino

if not dbDes.dialect.has_schema(dbDes, 'mfm'):
    dbDes.execute(CreateSchema('mfm'))

In [6]:
#Crea las tablas en la BBDD destino
metadata.create_all(dbDes)


In [7]:
#Copia las tablas que sean diccionarios a la base de datos destino
dfs = pd.read_excel('Tablas.xlsx',engine='openpyxl',header=None)
tablas_mfm=dfs.values
for tabla in tablas_mfm:
    if tabla[1] == 'D' :
        copiarTabla(dbOr,dbDes,string_to_table(tabla[0]))



In [8]:
#Cuenta el numero de sectores
tipo_zbs=["RU","UR","CA"]
res = dbOr.execute(text("select count(*) from mfm.mf_sector"))
nSectores = res.first()[0]
res.close()

sectores = np.zeros([len(tipo_zbs),nSectores])

In [9]:
#Cuenta el numero de zbs segun su tipo que tiene cada sector
q1="select * from (select sector_id , count(*) from mfm.mf_zbs where tipo = '"
q2="' group by sector_id) as zonas order by sector_id;"

for i in range(0,len(tipo_zbs)):
    res= dbOr.execute(text(q1+tipo_zbs[i]+q2))
    for r in res:
        sectores[i,r[0]-1]=r[1]
    res.close()

#print(sectores)

In [10]:
#Genera los sectores con sus zbs
NUMERO_DE_SECTORES_A_GENERAR=6
sectores_generados = np.random.multivariate_normal(np.mean(sectores,axis=1), np.cov(sectores), NUMERO_DE_SECTORES_A_GENERAR)
sectores_generados[sectores_generados<0]=0
sectores_generados=np.round(sectores_generados)
print(sectores_generados)

[[16.  2.  0.]
 [12.  2.  8.]
 [ 8.  0.  9.]
 [11.  2. 12.]
 [ 4.  0.  8.]
 [12.  1. 10.]]


In [11]:
#Cuenta el número zbs totales segun su tipo
q1="select count(*) from mfm.mf_zbs where tipo ='"
q2="';"
nTipo_zbs=[]
for i in range(0,len(tipo_zbs)):
    res= dbOr.execute(text(q1+tipo_zbs[i]+q2))
    nTipo_zbs.append(res.first()[0])
    res.close()
print(nTipo_zbs)

[77, 9, 37]


In [12]:
#Cuenta el numero de cias de AP segun su tipo en cada tipo de zbs
tipo_AP = ["MEDICINA FAMILIA", "PEDIATRIA AP"]
q1="select mz.code, coalesce(_count,0)  from (select zbs_cd as z , count(*) as _count from mfm.mf_cias_v2 where speciality_st = '"
q2="' group by zbs_cd) as s right join mfm.mf_zbs as mz on s.z=mz.code where mz.tipo='"
q3="' order by mz.code;"
cias=[]
for i in range(0,len(tipo_zbs)):
    cias_por_tipo = np.zeros([len(tipo_AP),nTipo_zbs[i]])
    for j in range(0,len(tipo_AP)):
        res= dbOr.execute(text(q1+tipo_AP[j]+q2+tipo_zbs[i]+q3))
        k=0
        for r in res:
            cias_por_tipo[j,k]=r[1]
            k+=1
        res.close()
    cias.append(cias_por_tipo)

In [13]:
#Genero los cias de AP en las zbs creadas
cias_generados=[]
for i in range(0,len(tipo_zbs)):
    cias_generado=np.random.multivariate_normal(np.mean(cias[i],axis=1), np.cov(cias[i]),np.sum(sectores_generados,axis=0).astype(int)[i] )
    cias_generado[cias_generado<0]=0
    cias_generado=np.round(cias_generado)
    cias_generados.append(cias_generado)
#print(cias_generados)


In [14]:
#Inserto en la BBDD destino los sectores,zbs y cias generados
i=0
idsZ=0
idsC=0
indice_tipos_cias=np.zeros([len(tipo_zbs)]).astype(int)
for sector in sectores_generados:
    s = gf.generar_sector(i)
    ins_sec = t_mf_sector.insert().values(s)
    dbDes.execute(ins_sec)
    for k in range(0,len(tipo_zbs)):
        j=0
        while j < sector[k]:
            zbs = gf.generar_zbs(idsZ,tipo_zbs[k],s)
            dbDes.execute(t_mf_zbs.insert().values(zbs))
            cias_g=cias_generados[k][indice_tipos_cias[k]]
            indice_tipos_cias[k]+=1
            l=0
            while l < len(tipo_AP):
                sp={"code": l, "st": tipo_AP[l]}
                m=0
                while m < cias_g[l]:
                    ci = gf.generar_cias(idsC,sp,s,zbs)
                    dbDes.execute(t_mf_cias_v2.insert().values(ci))
                    idsC+=1
                    m+=1
                l+=1
            idsZ+=1
            j+=1
    i+=1

In [17]:
dbDes.close()
dbDestino.dispose()
dbOr.close()
dbOrigen.dispose()